AI TRADER BOT por SHUBHAM THAPA 
https://www.kaggle.com/code/trooperog/ai-trading-bot

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas
from pandas_datareader import data as pdr
import yfinance as yfin

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# importing the libararies 

In [2]:
import numpy as np 
import pandas as pd 
import random 
import matplotlib.pyplot as plt 
import tensorflow as tf 
import math 
import pandas_datareader as pdr 
from datetime import datetime

from tqdm import tqdm_notebook , tqdm 
from collections import deque 

# building the AI trader network

In [3]:
class AI_Trader():
  
    def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
        self.state_size = state_size
        self.action_space = action_space
        self.memory = deque(maxlen=2000)
        self.inventory = []
        self.model_name = model_name
    
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_final = 0.01
        self.epsilon_decay = 0.995
    
        self.model = self.model_builder()
    def model_builder(self):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
        model.add(tf.keras.layers.Dense(units=64, activation='relu'))
        model.add(tf.keras.layers.Dense(units=128, activation='relu'))
        model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
        return model
  
    def trade(self, state):
    
        if random.random() <= self.epsilon:
            return random.randrange(self.action_space)
    
        actions = self.model.predict(state)
        return np.argmax(actions[0])
  
  
    def batch_train(self, batch_size):
    
        batch = []
        for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
            batch.append(self.memory[i])
      
        for state, action, reward, next_state, done in batch:
            reward = reward
            if not done:
                reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
            target = self.model.predict(state)
            target[0][action] = reward
      
            self.model.fit(state, target, epochs=1, verbose=0)
      
        if self.epsilon > self.epsilon_final:
            self.epsilon *= self.epsilon_decay
            
        

# data set preprocessing 

## sigmoid 

In [4]:
def sigmoid(x):
    return 1/(1 + math.exp(-x))

## price format function 

In [5]:
def stock_price_format(n):
    if n < 0 :
        return '- $ {:2f}'.format(abs(n))
    else :
        return '$ {:2f}'.format(abs(n))

# dataset loader 

In [6]:
def dataset_loader(stock_name):
    
    dataset = pdr.get_data_yahoo(stock_name, start='2022-10-24')
    
    start_date = str(dataset.index[0]).split()[0]
    end_date = str(dataset.index[-1]).split()[0]
    
    close = dataset['Close']
    return close  

In [7]:
import pandas
from pandas_datareader import data as pdr
import yfinance as yfin

yfin.pdr_override()

dataset_for_example = pdr.get_data_yahoo('AAPL', start='2022-10-24')

print(dataset_for_example)

[*********************100%***********************]  1 of 1 completed
                  Open        High         Low       Close   Adj Close   
Date                                                                     
2022-10-24  147.190002  150.229996  146.000000  149.449997  148.769241  \
2022-10-25  150.089996  152.490005  149.360001  152.339996  151.646088   
2022-10-26  150.960007  151.990005  148.039993  149.350006  148.669708   
2022-10-27  148.070007  149.050003  144.130005  144.800003  144.140442   
2022-10-28  148.199997  157.500000  147.820007  155.740005  155.030624   
...                ...         ...         ...         ...         ...   
2023-06-07  178.440002  181.210007  177.320007  177.820007  177.820007   
2023-06-08  177.899994  180.839996  177.460007  180.570007  180.570007   
2023-06-09  181.500000  182.229996  180.630005  180.960007  180.960007   
2023-06-12  181.270004  183.889999  180.970001  183.789993  183.789993   
2023-06-13  182.800003  184.149994  182.440

# state creator 

In [8]:
def state_creator(data, timestep, window_size):
    starting_id = timestep - window_size + 1
    if starting_id >= 0:
        windowed_data = data[starting_id:timestep+1]
    else:
        windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    state = []
    for i in range(window_size - 1):
        state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    return np.array([state])

# loading a dataset 

In [9]:
stock_name = 'AAPL'
data = dataset_loader(stock_name)

data 

[*********************100%***********************]  1 of 1 completed


Date
2022-10-24    149.449997
2022-10-25    152.339996
2022-10-26    149.350006
2022-10-27    144.800003
2022-10-28    155.740005
                 ...    
2023-06-07    177.820007
2023-06-08    180.570007
2023-06-09    180.960007
2023-06-12    183.789993
2023-06-13    183.309998
Name: Close, Length: 160, dtype: float64

# Training the AI trader

## setting the hyper parameters 

In [10]:
window_size = 10 
episodes = 1000

batch_size = 32
data_samples = len(data) - 1 


## defining the trader model 

In [11]:
trader = AI_Trader(window_size)

In [12]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                352       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


# training loop ( # this training process will take time )

In [ ]:
for episode in range(1, episodes + 1):
    print("Episode: {}/{}".format(episode, episodes))
    state = state_creator(data, 0, window_size + 1)
    total_profit = 0
    trader.inventory = []
  
    for t in tqdm(range(data_samples)):
        action = trader.trade(state)
        next_state = state_creator(data, t+1, window_size + 1)
        reward = 0
    
        if action == 1: #Buying
            trader.inventory.append(data[t])
            print("AI Trader bought: ", stock_price_format(data[t]))
      
        elif action == 2 and len(trader.inventory) > 0: #Selling
            buy_price = trader.inventory.pop(0)
      
            reward = max(data[t] - buy_price, 0)
            total_profit += data[t] - buy_price
            print("AI Trader sold: ", stock_price_format(data[t]), " Profit: " + stock_price_format(data[t] - buy_price) )
      
        if t == data_samples - 1:
            done = True
        else:
            done = False
      
        trader.memory.append((state, action, reward, next_state, done))
    
        state = next_state
    
        if done:
            print("########################")
            print("TOTAL PROFIT: {}".format(total_profit))
            print("########################")
    
        if len(trader.memory) > batch_size:
            trader.batch_train(batch_size)
      
    if episode % 10 == 0:
        trader.model.save("ai_trader_{}.h5".format(episode))

Episode: 1/1000


  0%|                                                                                     | 0/159 [00:00<?, ?it/s]

AI Trader bought:  $ 152.339996
AI Trader bought:  $ 149.350006
AI Trader bought:  $ 144.800003
AI Trader bought:  $ 155.740005
AI Trader sold:  $ 153.339996  Profit: $ 1.000000
AI Trader bought:  $ 150.649994
AI Trader bought:  $ 145.029999
AI Trader sold:  $ 138.380005  Profit: - $ 10.970001
AI Trader bought:  $ 138.919998
AI Trader bought:  $ 146.869995
AI Trader sold:  $ 150.039993  Profit: $ 5.239990
AI Trader bought:  $ 151.070007
AI Trader bought:  $ 148.110001
AI Trader sold:  $ 141.169998  Profit: - $ 14.570007
AI Trader bought:  $ 148.029999
AI Trader bought:  $ 148.309998
AI Trader bought:  $ 147.809998
AI Trader bought:  $ 146.630005
AI Trader bought:  $ 140.940002
AI Trader sold:  $ 142.649994  Profit: - $ 8.000000
1/1 [==============================] - 0s 11ms/step


2023-06-13 15:06:27.361404: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 0s 10ms/step


...

1/1 [==============================] - 0s 9ms/step
AI Trader bought:  $ 151.009995
1/1 [==============================] - 0s 9ms/step


 48%|████████████████████████████████████▎                                       | 76/159 [03:06<03:26,  2.49s/it]

1/1 [==============================] - 0s 9ms/step
AI Trader bought:  $ 153.850006
1/1 [==============================] - 0s 9ms/step


 48%|████████████████████████████████████▊                                       | 77/159 [03:08<03:26,  2.52s/it]

1/1 [==============================] - 0s 9ms/step
AI Trader bought:  $ 153.199997
1/1 [==============================] - 0s 9ms/step
